# Rad s geometrijama kroz korištenje Leafmapa i DuckDB

## Uvod

Ovaj Juypter Notebook demonstrira kako raditi sa geometrijama u DuckDB i Leafmapu

## Instalacija

Ukloni komentiranje u sljedećoj ćeliji kako bi instalirali potrebne Python libraryje

In [2]:
%pip install duckdb leafmap

Note: you may need to restart the kernel to use updated packages.


## Učitavanje librarya i konfiguracija

In [3]:
import duckdb
import leafmap

## Spajanje na DuckDB 

Možete se povezati ili s DuckDB-om u memoriji ili s DuckDB-om koji koristi datoteku kao pozadinsku bazu podataka. O uvom primjeru spajam se na pozadinsku bazu podataka koju sam stvorio na vlastitom računalu pod imenom "zagreb.db". Ukoliko se spajate u memoriji potrebno je ukloniti poveznicu na bazu i ostaviti argument (""). 

In [4]:
con = duckdb.connect("zagreb.db")

In [5]:
con.install_extension("spatial")
con.load_extension("spatial")

In [6]:
con.sql("SHOW TABLES;")

┌────────────────┐
│      name      │
│    varchar     │
├────────────────┤
│ amsterdam      │
│ cetvrti3765    │
│ cetvrti4326    │
│ stajalista     │
│ stajalista4765 │
│ četvrti        │
│ četvrti4765    │
│ četvrtiparquet │
└────────────────┘

## Vizualizacija podataka


Tablice koje su importirane u bazu "zagreb.db"su Geojson datoteke. U bazi imamo 8 tablica. U ovom primjeru radimo sa podacima stajališta tramvaja ZET-a i gradskim četvrtima Grada Zagreba. 


In [7]:
# kreiranje veze na bazu podataka
con = duckdb.connect('zagreb.db')

# kreiranje tablice i loadanje podataka 
con.sql('CREATE TABLE IF NOT EXISTS stajalista AS SELECT * FROM ST_Read(\'E:\\duck db shapefileovi\\tramvajska_stajalista_ZET.geojson\')')

# upit na tablicu koji stvara kolonu geom koja sadrži tip podatka geometry što je svojstvo Geojsona, shapefila...
con.table('stajalista').show()

┌──────────┬──────────────────────┬──────────────┬──────────────────────┬──────────────────────┬───────────────────────┐
│ OBJECTID │      Naziv_staj      │    Linije    │         Opis         │       Napomena       │         geom          │
│  int32   │       varchar        │   varchar    │       varchar        │       varchar        │       geometry        │
├──────────┼──────────────────────┼──────────────┼──────────────────────┼──────────────────────┼───────────────────────┤
│        1 │ Adžijina - istok     │ 2            │ smjer Črnomerc       │ NULL                 │ POINT (15.962046001…  │
│        2 │ Adžijina - zapad     │ 2            │ smjer Savska cesta   │ NULL                 │ POINT (15.961585 45…  │
│        3 │ Aleja javora - jug   │ 4,11,34      │ smjer Dubec          │ NULL                 │ POINT (16.069658 45…  │
│        4 │ Aleja javora - sje…  │ 4,11,34      │ smjer Kvaternikov …  │ NULL                 │ POINT (16.067935 45…  │
│        5 │ Arena Zagreb - isto



Koristite SQL upit za dohvaćanje podataka iz tablice "stajalista" iz DuckDB baze, uz isključivanje stupca "geom" i dodavanje novog stupca "geometry" koji sadrži tekstualni prikaz geometrije. 

In [9]:
stajalista_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM stajalista").df()
stajalista_df.head()

,OBJECTID,Naziv_staj,Linije,Opis,Napomena,geometry
0,1,Adžijina - istok,2,smjer Črnomerc,None,POINT (15.962046001000033 45.80759599900006)
1,2,Adžijina - zapad,2,smjer Savska cesta,None,POINT (15.961585 45.80778999900008)
2,3,Aleja javora - jug,"4,11,34",smjer Dubec,None,POINT (16.069658 45.82855699900006)
3,4,Aleja javora - sjever,"4,11,34",smjer Kvaternikov trg,None,POINT (16.067935 45.82886799900007)
4,5,Arena Zagreb - istok,"7,14,31",smjer Savski most,None,POINT (15.952866 45.77808499900004)


Koristite funkciju `leafmap.df_to_gdf()` kako biste pretvorili DataFrame `stajalista_df` u GeoDataFrame `stajalista_gdf`. Prilikom pretvaranja, određujete stupac "geometry" kao stupac koji sadrži geometrijske podatke.

In [10]:
stajalista_gdf = leafmap.df_to_gdf(stajalista_df, geometry='geometry', src_crs="EPSG:4326", dst_crs="EPSG:4326")

In [10]:
stajalista_gdf.explore()

In [8]:
cetvrti_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM četvrti").df()
cetvrti_df.head()

,RBR_GC,MBR_GC,IME_GC,IME_GO,IME_ZUPANI,SIFRA_PJ,sjediste_G,Shape_Leng,Shape_Area,geometry
0,17,00051,BREZOVICA,GRAD ZAGREB,GRAD ZAGREB,3.0,"Brezovica, Brezovièka cesta 100",71103.820664,1.273289e+08,MULTIPOLYGON (((15.920004866390428 45.74963045...
1,12,00108,DONJA DUBRAVA,GRAD ZAGREB,GRAD ZAGREB,3.0,"Zagreb, Dubrava 49",15418.388041,1.080980e+07,MULTIPOLYGON (((16.063200002676336 45.82973798...
2,1,00116,DONJI GRAD,GRAD ZAGREB,GRAD ZAGREB,3.0,"Zagreb, Ilica 25",9846.022151,3.016174e+06,MULTIPOLYGON (((16.001557120386938 45.81434387...
3,11,00132,GORNJA DUBRAVA,GRAD ZAGREB,GRAD ZAGREB,3.0,"Zagreb, Dubrava 49",39522.782154,4.026370e+07,MULTIPOLYGON (((16.054249779550105 45.93701794...
4,2,00159,GORNJI GRAD - MEDVE??AK,GRAD ZAGREB,GRAD ZAGREB,3.0,"Zagreb, Drakoviæeva ulica 15",18576.947222,1.019118e+07,MULTIPOLYGON (((15.988375060460246 45.85248711...


In [9]:
cetvrti_gdf = leafmap.df_to_gdf(cetvrti_df, geometry='geometry', src_crs="EPSG:4326", dst_crs="EPSG:4326")

In [10]:
cetvrti_gdf.explore()